In [ ]:
import sys
import glob
import pandas

# sys.path.append(f"/PATH_TO_KILT/KILT/")
import kilt.kilt_utils
import kilt.eval_downstream
import kilt.eval_retrieval
from importlib import reload 
kilt.eval_retrieval = reload(kilt.eval_retrieval)

K = [1,2,3,4,5,10,20,50,100]
rank_keys = ["url"]
results = {}

PATH_TO_PREDICTIONS = "YOUR PATH TO PREDICTIONS"# for example /private/home/USERNAME/side-internal/outputs/*/*/predictions/

guess_dataset_files = list(glob.glob(f"{PATH_TO_PREDICTIONS}/*.jsonl"))
gold_datasets_dir = f"/checkpoint/fabiopetroni/KILT/datasets-web/"

In [ ]:
try:
    print(len(results))
except:
    results = {}
    
for guess_dataset_file in guess_dataset_files:
    
    print(f"Evaluating: {guess_dataset_file}")

    experiment_id = guess_dataset_file.split("/")[-5]
    guess_file = guess_dataset_file.split("/")[-1]

    if experiment_id in results:
        continue
        
    gold_dataset = kilt.kilt_utils.load_data(f"{gold_datasets_dir}/{guess_file}")
    guess_dataset = kilt.kilt_utils.load_data(guess_dataset_file)

    # 0. validate input
    gold_dataset, guess_dataset = kilt.eval_downstream.validate_input(
        gold_dataset, guess_dataset
    )

    # 1. filter out ground thruth answers
    guess_dataset = kilt.eval_retrieval.filter_answers(guess_dataset)

    # 2. get retrieval metrics
    result = kilt.eval_retrieval.compute(gold_dataset, guess_dataset, K, rank_keys)
    
    results[experiment_id] = result
    

In [ ]:
df = pandas.DataFrame.from_dict(results).transpose().sort_index()
df = df[["precision@1", "success_rate@2", "success_rate@3", "success_rate@4", "success_rate@5", "success_rate@10", "success_rate@20", "success_rate@50", "success_rate@100"]]

df